In [1]:
import os
from typing import Optional, TypedDict, Annotated
from langchain_core.documents.base import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader, UnstructuredFileLoader
# from langchain_unstructured import UnstructuredLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool
import warnings
import dotenv

warnings.filterwarnings("ignore")

In [2]:
# .env파일 읽기
dotenv.load_dotenv()

# os라이브러리 이용한 읽기
# os.environ['OPENAI_API_KEY'] = "API-Key"
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

True

In [3]:
# LLM Load

llm = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0.)

In [4]:
llm.invoke("1+3은?")

AIMessage(content='1 + 3은 4입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8fde57a-68b4-4d07-8eea-7ff864a325c1-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
prompt = ChatPromptTemplate.from_messages([
    # MessagesPlaceholder(variable_name="context"),
    ("system", 
    """

    context : {context}
    
    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    규칙 1. 주어진 context만을 이용하여 답변해야합니다. 
    규칙 2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    규칙 3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    규칙 4. 항상 친절한 말투로 응대합니다.
    규칙 5. 웹사이트 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요.
    """),
    ("human", "{query}")
])

In [6]:
# Document Embedding

def embedding_file(file="Practice_document.pdf"):
    print("Embedding 시작")
    Embeddings = OpenAIEmbeddings()
    print("Embedding 완료!\n파일 로딩중")
    loader = UnstructuredFileLoader(file_path=f"./files/{file}")
    print("파일 로딩 완료!\n텍스트 스플리터 생성")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=["\n\n"])
    split_docs = loader.load_and_split(text_splitter)
    print("문서 분할 완료")
    vector_store = FAISS.from_documents(split_docs, Embeddings)
    print("벡터 저장소 생성 완료")

    retriever = vector_store.as_retriever()
    print("벡터 리트리버 생성 완료")

    return retriever

In [ ]:
Embeddings = OpenAIEmbeddings()

In [ ]:
loader = UnstructuredFileLoader(file_path=f"./files/{file}")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=["\n\n"])

In [ ]:
split_docs = loader.load_and_split(text_splitter)

In [ ]:
vector_store = FAISS.from_documents(split_docs, Embeddings)

In [ ]:
retriever = vector_store.as_retriever()

In [7]:
retriever = embedding_file()

Embedding 시작
Embedding 완료!
파일 로딩중
파일 로딩 완료!
텍스트 스플리터 생성


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


문서 분할 완료
벡터 저장소 생성 완료
벡터 리트리버 생성 완료


In [8]:
docs = retriever.invoke("개강하는 날짜가 언제야?")

In [9]:
docs

[Document(id='0e9fded8-e503-43b7-bc18-52d91ceae30d', metadata={'source': './files/Practice_document.pdf'}, page_content='교육 기간\n\n2024. 09. 25 - 25. 03. 21, 약 6개월이며 120일 과정입니다.\n\n교육 시간\n\n평일 10:00 - 18:00, 주말은 운영하지 않습니다.\n\n교육 환경\n\n100% 실시간 온라인 과정 (타 KDT훈련 및 직장 병행 불가)\n\n홈페이지\n\n모두의연구소 홈페이지 (modulabs.co.kr)\n\n아이펠 데이터사이언티스트 홈페이지 (ds.aiffel.io)\n\n주소\n\n모두의연구소 강남 캠퍼스 (서울 강남구 강남대로 324 역삼디오슈페리움 2층 모두의연구소) https://naver.me/FslREXGR'),
 Document(id='26b58c64-711c-45fa-89cd-b23e765cc858', metadata={'source': './files/Practice_document.pdf'}, page_content='아이펠 데이터사이언티스트 3기\n\n1\n\nA4. 내일배움카드 실수령 후 다음 날 훈련생 등록이 가능하여 개강일 전일(평일 기준)까지 발급이 가능해야 합니다. 가능한 아이펠 서류제출과 함께 바로 내일배움카드 신청을 해 주시는 게 가장 안전합니다! 부득이 하게 발급이 늦어지는 경우, 교육 시작 후 4일 까지 실수령 하여야 최종 등록 후 수강이 가능이 가능하니 미리 발급받고 편하게 입학 준비하세 요! (*개강일 이후 등록이 진행 되는 경우, 결석 또는 지각 처리 될 수 있습니다.)\n\nQ5. 지원하는데 나이 제한이 있나요?'),
 Document(id='99867b0b-9991-4af3-a8fe-2c8643bd5446', metadata={'source': './files/Practice_document.pdf'}, page_content='Q16. 졸업 프로젝트로는 어떤 걸 할 

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page)

In [14]:
chain = {"context":retriever | RunnableLambda(lambda docs:'\n\n'.join(x.page_content for x in docs)),\
         "query":RunnablePassthrough()}| prompt | llm

In [17]:
result = chain.invoke("수강료는 얼마인가요?")

In [18]:
result

AIMessage(content='내일배움카드 발급이 가능하다면 100% 무료로 입학이 가능합니다. 단, 잔액이 1원이라도 있어야 하니 꼭 확인해주세요! 더 궁금한 점이 있으시면 언제든지 문의해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 775, 'total_tokens': 832, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4916fb9-3d2f-46b5-b954-681226b08222-0', usage_metadata={'input_tokens': 775, 'output_tokens': 57, 'total_tokens': 832, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})